In [25]:
import networkx as nx
import random
from easygui import *
import math
import time
import sys
import os
from termcolor import colored

In [26]:
CLASSES_PATH = os.path.dirname(os.path.abspath('../Classes/'))
if not (CLASSES_PATH in sys.path):
    sys.path.append(CLASSES_PATH)
from Classes.Files_Handler_Class import Files_Handler
from Classes.K_Shell_Calculate_Class import K_Shell_Calculate
from Classes.Resd_Network_Infos_Class import Resd_Network_Infos
from Classes.SIR_Diffusion_Model_Class import SIR_Diffusion_Model
from Classes.Get_Past_Results_Class import Get_Past_Results
from Classes.Network_Infos_Writer_Class import Network_Infos_Writer
from Classes.Layers_Ranking_Class import Layers_Ranking
from Classes.Network_Node_Centrality_Class import Network_Node_Centrality
from Classes.Bcolors_Class import Bcolors as bcolors
from Classes.Random_Nodes_Selection_Class import Random_Nodes_Selection

In [27]:
color_list = ["light_red", "light_green", "light_yellow",
               "light_blue","light_magenta", "light_cyan",
               "blue", "red", "white", "green", "yellow",
                 "magenta", "cyan", ]

In [28]:
files_handler_obj = Files_Handler()
file_path = files_handler_obj.select_files("text files", ".edgeslist .edgelist .edges .mtx .txt")
if file_path is None or file_path == '':
    sys.exit("File Selection Canceled !")
file_info = files_handler_obj.get_file_path_info(file_path)
network_name = file_info['name']
network_type = file_info['type']
network_path = file_info['path']
if network_name == "":
    sys.exit("Dont Network Selection!")
file_info


{'path': 'D:/Masters thesis/Networks Dataset/Multilayer Dataset/00 - General Forms/co author/T_F/',
 'name': 'T_F',
 'type': '.edgeslist'}

In [29]:
resd_network_infos_object = Resd_Network_Infos()
(
    network_layers_info,
    network_layers_nodes,
    entra_layer_edges,
    entra_layer_edges_features,
    inter_layer_edge,
) = Resd_Network_Infos.read_nodeFrom_layerFrom_nodeTo_layerTo(
    network_path, network_name, network_type
)


In [ ]:
print(file_info['name'], '\n')
layers_id = []
network_layers_count = len(network_layers_info)
graphs_of_network = [None] * network_layers_count
network_entier_edges = ""
layers_nodes_infect_scale = []

i = 0
j = 0
while i < network_layers_count:
    graphs_of_network[i] = nx.Graph()
    network_layers_nodes[i] = list(set(network_layers_nodes[i]))
    layers_nodes_infect_scale.append({})
    if len(network_layers_nodes[i]) > 0:
        # graphs_of_network[i].add_nodes_from(network_layers_nodes[i])
        graphs_of_network[i].add_edges_from(entra_layer_edges[i])

        nx.set_node_attributes(graphs_of_network[i], None, "degree")
        nx.set_node_attributes(graphs_of_network[i], None, "k_shell")
        nx.set_node_attributes(graphs_of_network[i], None, "k_shell_itr")
        nx.set_node_attributes(graphs_of_network[i], None, "nip")
        nx.set_node_attributes(graphs_of_network[i], None, "sombor_index")
        nx.set_node_attributes(graphs_of_network[i], None, "ego_density")
        nx.set_node_attributes(graphs_of_network[i], None, "ego_degree")
        nx.set_node_attributes(graphs_of_network[i], None, "ego_k_shell")
        nx.set_node_attributes(graphs_of_network[i], None, "ego_degree_mean")
        nx.set_node_attributes(graphs_of_network[i], None, "kss")
        nx.set_node_attributes(graphs_of_network[i], 1, "vote_power")
        nx.set_node_attributes(graphs_of_network[i], None, "clustering")
        nx.set_node_attributes(graphs_of_network[i], None, "SIR")
        

        graphs_of_network[i].graph["id"] = i
        graphs_of_network[i].graph["layer_density"] = nx.density(graphs_of_network[i])
        graphs_of_network[i].graph["layer_degree_histogram"] = None
        graphs_of_network[i].graph["layer_edge_weight"] = None
        graphs_of_network[i].graph["layer_sombor_index"] = None
        graphs_of_network[i].graph["layer_nodes_weight"] = None
        graphs_of_network[i].graph["layer_k_shell_weight"] = None

        K_Shell_Calculate_Object = K_Shell_Calculate(graphs_of_network[i])
        graphs_of_network[i] = K_Shell_Calculate_Object.get_k_shell_info()
        del K_Shell_Calculate_Object
        degrees = dict(graphs_of_network[i].degree())
        nx.set_node_attributes(graphs_of_network[i], degrees, "degree")
        layers_id.append(str(i))
        print(colored("Layer "  + str(i) + ": " + str(graphs_of_network[i].number_of_nodes()) + " Node And " +
                       str(graphs_of_network[i].number_of_edges()) + " Edge", color_list[j]))
        # print(colored(graphs_of_network[i].graph['k_shell_info'], color_list[i]))
    i += 1
    j += 1
    if j >= len(color_list):
        j = 0

network_entier_nodes_list = []
for item in network_layers_nodes:
    network_entier_nodes_list += item

network_entier_nodes_list = list(set(network_entier_nodes_list))
network_entier_nodes_count = len(network_entier_nodes_list)
print()
print("network entier nodes : " + colored(str(network_entier_nodes_count), "yellow"))

T_F 

Layer 1: 44832 Node And 1161114 Edge
Layer 2: 48089 Node And 11165818 Edge

network entier nodes : 48089


In [31]:
del network_layers_nodes, entra_layer_edges
del entra_layer_edges_features

In [32]:
random_nodes_selection_obj = Random_Nodes_Selection()
network_infos_writer_object = Network_Infos_Writer(network_path, network_name)
SIR_diffusion_model_object = SIR_Diffusion_Model()

In [33]:
node_fraction = 0.1
node_fraction = random_nodes_selection_obj.get_node_fraction(network_entier_nodes_count)
print(network_name, '\n', math.ceil(network_entier_nodes_count * node_fraction))

T_F 
 9618


In [34]:
random_selected_nodes = []
population = []

In [35]:
print(network_name, '\n')
f_p = '{:12.5f}' # float padding
i_p = '{:12.0f}' # float padding

beta = 0.01
landa = 0.7
epoch = 1000

calced_sir_nodes_count = 0
get_past_results_obj = Get_Past_Results(network_path, network_name)
nodes_result_status, nodes_result = get_past_results_obj.get_past_node_att_results(graphs_of_network)
calced_sir_nodes_count = len(nodes_result.keys())
random_selected_nodes.extend(list(nodes_result.keys()))
random_selected_nodes = list(set(random_selected_nodes))
random_selected_nodes_count = len(random_selected_nodes)
if nodes_result_status:
    print('nodes result status from graph file:', bcolors.HEADER + bcolors.OKGREEN + str(nodes_result_status) + f' ({random_selected_nodes_count})' + bcolors.ENDC, '\n')
else:
    print('nodes result status from graph file:', bcolors.HEADER + bcolors.FAIL + str(nodes_result_status) + bcolors.ENDC, '\n')

# if nodes_result_status == False:
nodes_result = get_past_results_obj.get_past_SIR_results_from_temp_file(beta, landa, epoch)
calced_sir_nodes_count = len(nodes_result.keys())
random_selected_nodes.extend(list(nodes_result.keys()))
random_selected_nodes = list(set(random_selected_nodes))
if len(nodes_result) > 0 :
    nodes_result_status = True
    print('nodes result status from temp SIR  file:', bcolors.HEADER + bcolors.OKGREEN + str(nodes_result_status) + f' ({len(random_selected_nodes) - random_selected_nodes_count})' + bcolors.ENDC, '\n')
else:
    print('nodes result status from temp SIR  file:', bcolors.HEADER + bcolors.FAIL + str(False) + bcolors.ENDC, '\n')
random_selected_nodes_count = len(random_selected_nodes)
if calced_sir_nodes_count > 0:
    for k, v in nodes_result.items():
        for layer, sir_val in v.items():
            graphs_of_network[layer].nodes[k]["SIR"] = sir_val

    for k, v in nodes_result.items():
        for i, graph in enumerate(graphs_of_network):
            if graph.number_of_nodes() > 0 and k in graph:
                sir_val = graph.nodes[k]["SIR"]
                if sir_val is None:
                    graph.nodes[k]["SIR"] = SIR_diffusion_model_object.biased_SIR_monoplex_with_seed_set_model(
                        network=graph, seed=[k], beta=beta, landa=landa, iteration=epoch, bias=5)    
                    network_infos_writer_object.write_node_SIR(beta=beta, landa=landa,
                                                            epoch=epoch,layer=j,
                                                              node=k, sir_value=graph.nodes[k]["SIR"]) 

message = (f"calced sir nodes count: {bcolors.OKGREEN + str(len(random_selected_nodes)) + bcolors.ENDC}")
print(message)

T_F 

nodes result status from graph file: True (202) 

nodes result status from temp SIR  file: True (0) 

calced sir nodes count: 202


In [36]:
for node in random_selected_nodes:
    for graph in graphs_of_network:
        if node in graph:
            network_infos_writer_object.write_node_SIR(beta=beta, landa=landa,
                                                            epoch=epoch,layer=graph.graph['id'],
                                                              node=node, sir_value=graph.nodes[node]["SIR"]) 

In [37]:
degree_mean = 0
for node in network_entier_nodes_list:
    node_degree_sum = 0
    layers_counter = 0
    for graph in graphs_of_network:
        if graph.number_of_nodes() > 0 and node in graph:
            node_degree_sum += graph.degree[node]
            layers_counter += 1
    degree = math.ceil(node_degree_sum / layers_counter)
    degree_mean += degree
    if degree >= len(population):
        dist = degree - len(population)
        i = 0
        while i <= dist:
            population.append([])
            i += 1
    population[degree].append(node)
degree_mean = degree_mean / graph.number_of_nodes()
print("degree mean:", degree_mean)
text = "calced sir nodes count: " + str(len(random_selected_nodes)) + "\n\nSelect your selection strategy"
title = network_name
choices = ["Select nodes with bias higher than average degree", 
        "Only select nodes with higher and neighborhood average degree",
        "Only select nodes with neighborhood average degree",
            "Only select nodes with high degree"]
with files_handler_obj.tk_windows_timer(timeout=15):
    output = choicebox(text, title, choices)
if not(output in choices):
    output = choices[0]
print(bcolors.WARNING + '\n' + output + '\n' + bcolors.ENDC)
temp_random_selected_nodes = random_nodes_selection_obj.get_random_nodes_selection(output, choices, population, degree_mean, node_fraction)
temp_random_selected_nodes.extend(random_selected_nodes)
temp_random_selected_nodes = list(set(temp_random_selected_nodes))
temp_random_selected_nodes_count = len(temp_random_selected_nodes)
# min_degree_arr = []
# max_degree_arr = []
# for graph in graphs_of_network:
#     if temp_random_selected_nodes[-1] in graph:
#         min_degree_arr.append(graph.degree[temp_random_selected_nodes[-1]])
#     if temp_random_selected_nodes[0] in graph:
#         max_degree_arr.append(graph.degree[temp_random_selected_nodes[0]])
# print("min dgree: ",min_degree_arr , "\nmax dgree: ", max_degree_arr)

degree mean: 257.3167876229491

Select nodes with bias higher than average degree



In [38]:
message = (f"Selected Nodes Count: {temp_random_selected_nodes_count}\n\ncalced sir nodes count: {bcolors.OKGREEN + str(random_selected_nodes_count) + bcolors.ENDC}\n\nnew nodes: {bcolors.FAIL + str(temp_random_selected_nodes_count - random_selected_nodes_count) + bcolors.ENDC}")
print(message)
message = (f"Selected Nodes Count: {temp_random_selected_nodes_count}\n\ncalced sir nodes count: {random_selected_nodes_count}\n\nnew nodes: {temp_random_selected_nodes_count - random_selected_nodes_count}")
del population

Selected Nodes Count: 13533

calced sir nodes count: 202

new nodes: 13331


In [39]:
print(network_name, '\n', "Layer count: ", len(graphs_of_network)-1)
print("network entier nodes count:", network_entier_nodes_count)
new_sir_calc = False
with files_handler_obj.tk_windows_timer(timeout=15):
    try:
        new_sir_calc = ynbox(message, f'{network_name} New nodes SIR calc')
    except:
        new_sir_calc = False

if new_sir_calc:
    random_selected_nodes.extend(temp_random_selected_nodes)
    random_selected_nodes = list(set(random_selected_nodes))
    print("\t\t\tSelected Nodes Count: ", str(temp_random_selected_nodes_count - random_selected_nodes_count),
          " + ", calced_sir_nodes_count, " = ", len(random_selected_nodes))
    print()
    print('\tCounter\t\tNode\t\tLayer\t\tdegree\t\tSIR\t\tCompution Time(s)')
    c = 0
    for i, node in enumerate(random_selected_nodes):
        for j, graph in enumerate(graphs_of_network):
            if node in graph:
                node_infect_scale = graph.nodes[node]["SIR"]
                start_time = time.time()
                if node_infect_scale is None:
                    node_infect_scale = SIR_diffusion_model_object.biased_SIR_monoplex_with_seed_set_model(
                        network=graph, seed=[node], beta=beta, landa=landa, iteration=epoch, bias=5)
                graph.nodes[node]["SIR"] = node_infect_scale
                finish_time = time.time()
                network_infos_writer_object.write_node_SIR(beta=beta, landa=landa,
                                                            epoch=epoch,layer=j,
                                                              node=node, sir_value=node_infect_scale)
                print(colored((i_p.format(i + 1)+ '\t\t' +
                               str(node)+ ' \t' +
                               i_p.format(j) + '\t\t' +
                               str(graph.degree[node])+ '\t\t ' +
                               str(f_p.format(node_infect_scale)) + '\t' +
                               str(f_p.format(finish_time - start_time))),color_list[c]))
                c += 1
                if c >= len(color_list):
                    c = 0
                layers_nodes_infect_scale[j][node] = node_infect_scale
            else:
                layers_nodes_infect_scale[j][node] = 0
        print()
else:
    # random_selected_nodes = list(set(list(nodes_result.keys())))
    for k, v in nodes_result.items():
        for layer, sir_val in v.items():
            graphs_of_network[layer].nodes[k]["SIR"] = sir_val
            # network_infos_writer_object.write_node_SIR(beta=beta, landa=landa,
            #                                     epoch=epoch, layer=layer,
            #                                      node=k, sir_value=sir_val)

print(colored("\t\t\t    ______________", 'light_yellow') + colored(" Finished ", 'light_green') + colored("______________", 'light_yellow'))
# del SIR_diffusion_model_object

T_F 
 Layer count:  2
network entier nodes count: 48089
			    ______________ Finished ______________


In [40]:
import numpy as np
random_selected_nodes_copy = random_selected_nodes.copy()
for node in random_selected_nodes_copy:
    for graph in graphs_of_network:
        if node in graph:
            sir_value = graph.nodes[node]['SIR']
            if type(sir_value) is np.float64:
                sir_value = float(sir_value)
                graph.nodes[node]['SIR'] = sir_value
            if not((type(sir_value) is float) and (sir_value > 0)):
                print(node, sir_value, type(sir_value))
                node_infect_scale = SIR_diffusion_model_object.biased_SIR_monoplex_with_seed_set_model(
                        network=graph, seed=[node], beta=beta, landa=landa, iteration=epoch, bias=5)
del random_selected_nodes_copy

In [41]:
if new_sir_calc:
    new_node_fraction = float("%.3f" % (len(random_selected_nodes) / network_entier_nodes_count))
else:
    new_node_fraction = node_fraction
print(bcolors.FAIL +  str(node_fraction) + bcolors.ENDC, bcolors.OKGREEN + str(new_node_fraction) + bcolors.ENDC)

0.2 0.2


In [42]:
network_infos_writer_object.wirte_layers_mean_SIR_infos_csv(new_node_fraction, beta, landa, epoch, graphs_of_network, random_selected_nodes)

In [43]:
print(network_name)
layer_past_result_status = get_past_results_obj.get_past_layer_att_results(graphs_of_network)
if layer_past_result_status:
        print("Load layer results: ", bcolors.HEADER + bcolors.OKGREEN + str(layer_past_result_status) + bcolors.ENDC)
else:
        print("Load layer results: ", bcolors.HEADER + bcolors.FAIL + str(layer_past_result_status) + bcolors.ENDC)
if not layer_past_result_status:
        layers_ranking_object = Layers_Ranking()
        print(bcolors.HEADER + bcolors.OKBLUE + 'Calc layer layers_density_weight' + bcolors.ENDC)
        layers_ranking_object.layers_density_weight(graphs_of_network)
        print(bcolors.HEADER + bcolors.OKBLUE + '\nCalc layer layers_degree_distribution' + bcolors.ENDC)
        layers_ranking_object.layers_degree_distribution(graphs_of_network)
        print(bcolors.HEADER + bcolors.OKBLUE + '\nCalc layer layers_edges_and_sombor_index' + bcolors.ENDC)
        layers_ranking_object.layers_edges_and_sombor_index(graphs_of_network)
        print(bcolors.HEADER + bcolors.OKBLUE + '\nCalc layer layers_nodes_weight' + bcolors.ENDC)
        layers_ranking_object.layers_nodes_weight(graphs_of_network)
        print(bcolors.HEADER + bcolors.OKBLUE + '\nCalc layer layers_k_shell_weight' + bcolors.ENDC)
        layers_ranking_object.layers_k_shell_weight(graphs_of_network)

T_F
Load layer results:  True


In [44]:
# if not layer_past_result_status:
network_infos_writer_object.write_network_layer_infos_csv(graphs_of_network)

In [45]:
print('Layer\t density\t degree_histogram\t edge_weight\t sombor_index\t nodes_weight\t k_shell_weight')
f_p = '{:9.7f}' # float padding
c = 0
for i, graph in enumerate(graphs_of_network):
    if graph.number_of_nodes() > 0:
        print(colored((str(i)+ ' \t ' + str(f_p.format(graph.graph["layer_density"]))+ ' \t ' +
                       str(f_p.format(graph.graph["layer_degree_histogram"]))+ ' \t\t ' +
                       str(f_p.format(graph.graph["layer_edge_weight"]))+ ' \t ' +
                       str(f_p.format(graph.graph["layer_sombor_index"]))+ ' \t ' +
                       str(f_p.format(graph.graph["layer_nodes_weight"]))+ ' \t ' +
                       str(f_p.format(graph.graph["layer_k_shell_weight"]))),
                       color_list[c]))
    c += 1
    if c >= len(color_list):
        c = 0

Layer	 density	 degree_histogram	 edge_weight	 sombor_index	 nodes_weight	 k_shell_weight
1 	 0.0011554 	 51.7984475 		 1161114.0000000 	 2342.5532064 	 44832.0000000 	 45.7239210
2 	 0.0096569 	 464.3813762 		 11165818.0000000 	 3187.2066430 	 48089.0000000 	 1988.5527167


In [46]:
print(network_name)
past_results_status, past_results_nodes = get_past_results_obj.read_nodes_att_from_temp_graph(graphs_of_network)
if past_results_status:
    print("\nLoad temp results: ", bcolors.HEADER + bcolors.OKGREEN + str(past_results_status) + bcolors.ENDC)
    print("Load temp results count: ", bcolors.HEADER + bcolors.OKGREEN + str(len(past_results_nodes)) + bcolors.ENDC, '\n')
    random_selected_nodes = past_results_nodes
else:
    print("Load temp results: ", bcolors.HEADER + bcolors.FAIL + str(past_results_status) + bcolors.ENDC, '\n')
Network_Node_Centrality_obj = Network_Node_Centrality()
hop_num = 2
Network_Node_Centrality_obj.get_nodes_centrality(network_infos_writer_object, graphs_of_network, random_selected_nodes, hop_num)


T_F

Load temp results:  True
Load temp results count:  202 



100%|██████████| 202/202 [00:00<00:00, 12557.24 Node/s]


In [47]:
for node in random_selected_nodes:
    for graph in graphs_of_network:
        if graph.number_of_nodes() > 0 and node in graph:
            graph.nodes[node]["degree"]  = graph.degree(node)
            graph.nodes[node]["layer_density"] =  graph.graph['layer_density']
            graph.nodes[node]['layer_degree_histogram'] = graph.graph["layer_degree_histogram"]
            graph.nodes[node]['layer_edge_weight'] = graph.graph["layer_edge_weight"]
            graph.nodes[node]['layer_sombor_index'] = graph.graph["layer_sombor_index"]
            graph.nodes[node]['layer_nodes_weight'] = graph.graph["layer_nodes_weight"]
            graph.nodes[node]['layer_k_shell_weight'] = graph.graph["layer_k_shell_weight"]


In [48]:
print(network_name)
network_infos_writer_object.write_network_nodes_info_csv(node_fraction, graphs_of_network, random_selected_nodes)


T_F
